# MODELOS COMPUTACIONALES II: Proyecto Final
### Vladimir Abisai Espinosa Torrijos
El proyecto debe desarrollar las habilidades aprendidas a lo largo del semestre: 
- Analisis de dataset
- Identificacion del tipo de problema (clasificacion o regresion)
- Creacion de propuestas de modelos (que modelo supervisado utilizar, como crear la ANN o CNN a utilizar)
- Evaluacion de resultados acorde al problema (clasificacion: accuracy, precision, recall, f1; regresion: mse, rmse, r^2, etc)
- Analisis de resultados (determinar el mejor resultado posible y justificar por que es el mejor)
- El resultado debe de ser interpretable

## Nota: El proyecto lo realize en Pycharm, por lo que no contiene la forma habitual de un notebook de Jupyter (este archivo solo lo hice para subirlo al repositorio con la explicacion del codigo). Dado esto, las imagenes de los resultados (graficos) solo se pueden visualizar si el codigo se ejecuta en pycharm, ademas las subi a drive por si desea verlos sin ejecutar el codigo: 
## https://drive.google.com/drive/folders/1q6L2OGNPsHmLaEjq0t7iyTeysCsa2w95?usp=sharing

## Las preguntas del proyecto las contesto al final del archivo ☻

In [ ]:
#siempre importo todas las librerias y funciones al comienzo, no me gusta colocarlas en medio del codigo.
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier

# la ruta de acceso para el dataset completo :D
dataset_path = r"C:\Users\vladi\OneDrive\Documentos\dataset_proyecto_MII\dataset"

# tamaño al que se redimensionan las imágenes. Este paso es mega importante porque la maquina debe aprender en base a todas las imagenes, y si tienen tamaños diferentes esto seria fatal.
IMG_SIZE = 128

# dict de las etiquetas
labels_dict = {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
label_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# cargue las imágenes con una funcion nomas para que fuera mas limpio el codigo y no tener todo de manera 100% lineal y desordenada. Funcinaria pero no me gusta.
def load_images(folder):
    data = []
    labels = []
    for category in labels_dict:
        folder_path = os.path.join(folder, category)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                labels.append(labels_dict[category])
            except:
                continue
    return np.array(data), np.array(labels)

train_data, train_labels = load_images(os.path.join(dataset_path, 'Training'))
test_data, test_labels = load_images(os.path.join(dataset_path, 'Testing'))

# normalizar
train_data = train_data / 255.0
test_data = test_data / 255.0
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

# One-hot encoding: lo utilize para evitar usar las etiquetas a modo str y mejor usar numeros :D
train_labels_cat = to_categorical(train_labels, num_classes=4)
test_labels_cat = to_categorical(test_labels, num_classes=4)

# aqui puse como se distribuyen las clases en las carpetas, lo podriamos hacer de manera manual simplemente llendo a las carpetas, pero crei mejor poder verlo mediante el codigo.
# ademas, en unos videos que utilize para el proyecto se menciona que nos permite ver en un grafico de manera mas comoda si hay mas imagenes de una categoria o otra.
plt.figure(figsize=(8,4))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # azul, naranja, verde, rojo

counts = [np.sum(train_labels == i) for i in range(4)]
plt.figure(figsize=(8, 4))
sns.barplot(x=label_names, y=counts, palette=colors)
plt.title("Distribucion de clases en el set de entrenamiento", fontsize=14)
plt.xlabel("Clase", fontsize=12)
plt.ylabel("Número de imágenes", fontsize=12)
plt.tight_layout()
plt.show()

# hacemos la cnn: lo hice de solo 10 epocas porque la verdad era demasiado tardado con mas (lo hice con 50 pero demoro mucho)
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

cnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.summary()

cnn_model.fit(train_data, train_labels_cat, epochs=10, batch_size=32,
              validation_split=0.2)

# evaluamos la red cnn
cnn_preds = np.argmax(cnn_model.predict(test_data), axis=1)
print("=== CNN Classification Report ===")
print(classification_report(test_labels, cnn_preds, target_names=label_names))

# MODELO RANDOM FOREST
# Aplanar imágenes para modelo clásico
train_data_flat = train_data.reshape(len(train_data), -1)
test_data_flat = test_data.reshape(len(test_data), -1)

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(train_data_flat, train_labels)
rf_preds = rf_model.predict(test_data_flat)

print("♦♦♦ Informe de clasificacion de Random Forest ♦♦♦")
print(classification_report(test_labels, rf_preds, target_names=label_names))


# Matrices de confusion
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.heatmap(confusion_matrix(test_labels, cnn_preds), annot=True, fmt='d', cmap='plasma')
plt.title('Matriz de confucion de la CNN ☻')
plt.xticks(ticks=[0.5,1.5,2.5,3.5], labels=label_names)
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=label_names)

plt.subplot(1,2,2)
sns.heatmap(confusion_matrix(test_labels, rf_preds), annot=True, fmt='d', cmap='magma')
plt.title('Matriz de confusion del Random Forest ☻')
plt.xticks(ticks=[0.5,1.5,2.5,3.5], labels=label_names)
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=label_names)
plt.tight_layout()
plt.show()


### 1) Analisis del dataset
Para nuestro caso particular he decidido utilizar el dataset de brain tumor de 4 categorias (no confie en buscar otro de la red por cuestiones de tiempo), el dataset contiene 7,023 imagenes en formato .jpg, organizadas en una jerarquia de carpetas dentro de dos principales subcarpetas: la de  “Training” y “Testing”, que a su vez contienen mas subcarpetas para cada clase (glioma, meningioma, notumor y pituitary). Cada carpeta representa una categoría correspondiente al tipo de imagen de resonancia magnetica. Una de las tareas claves del analisis fue analizar primero la distribución de clases en el conjunto de entrenamiento, lo cual se visualizó mediante un gráfico de barras de colores diferenciados (algo recomendado en varias fuentes). 
Esta sencilla grafica nos permitio confirmar que las clases estan distribuidas de forma relativamente balanceada, aunque con ligeras diferencias en la cantidad de imagens por categoria. Esto es crucial para entender posibles sesgos en el dataset, ya que una desproporcion marcada entre clases podria afectar el rendimiento y la imparcialidad de los modelos predictivos. Ademas, se implemento una funcion para cargar, redimensionar y preprocesar las imagenes, facilitando su posterior uso en ls modelos de aprendizaje. La redimensión a un tamaño uniforme asegura que todos los modelos puedan procesar las imagenes eficientemente y sin errores por diferencias de forma.

### 2) Identificacion del tipo de problema (clasificacion o regresion)
Al analizar como se estructura el dataset, donde cada imagen esta asociada a una etiqueta que indica su clase correspondiente (glioma, meningioma, notumor, pituitary), es muy evidente que estamos frente a un problema de clasificacion supervisada (de muchas clases: multiclase). En este tipo de problemas, el objetivo del modelo es aprender patrones discriminativos que le permitan predecir correctamente la clase de una nueva imagen no vista previamente (cabe señalar que aqui radica la importancia de dividir los datos en grupos de entrenamiento/prueba, ya que de no hacerlo podemos caer en el underfitting u overfitting, lo cual no haria generalizable nuestro proyecto). A diferencia de los problemas de regresion, donde la variable objetivo es continua (o mejor dicho “numerica”), en este caso la variable objetivo es una categoria (se trata de una cualidad y no de un numero). La naturaleza supervisada del problema implica que contamos con ejemplos etiquetados durante el entrenamiento, lo cual permite entrenar modelos con retroalimentacion directa sobre sus predicciones.

### 3) Creacion de Propuestas de Modelos Supervisados y su Implementación
Ya una vez que fue identificado el tipo de problema, se procedio a seleccionar e implementar modelos supervisados adecuados para la tarea de la clasificacion de las imagenes. En este caso comparamos dos modelos: uno basado en aprendizaje profundo mediante la Red neuronal convolucional (CNN), y otro basado en aprendizaje automatico tradicional mediante un clasificador random forest. 
Sabemos que las CNN estan diseñadas específicamente para procesar imagenes, gracias a su capacidad de extraer caracteristicas espaciales mediante diferentes capas y filtros. La estructura de la red consiste en multiples capas, funciones ReLU, capas de pooling para reducir la dimensionalidad, y capas densas al final para realizar la clasificacion. Por el otro lado, el modelo random forest (aunque no fue diseñado originalmente para imagenes) se utilizp como un punto de comparacion luego de transformar las imagenes a vectores planos. Esto permitio evaluar la diferencia en desempeño entre las CNN y los modelos mas generales (como el random forest), y tambien mostro como distintas metodologias pueden ser aplicadas al mismo problema de clasificacion. Ambos modelos fueron entrenados utilizando el conjunto de entrenamiento y luego evaluados en el conjunto de prueba
### 4) Evaluacion de Resultados Acorde al Problema
Las metricas de evaluacion fueron: accuracy, precision, recall y f1-score. 
No las coloque en ningun grafico en especial, pero es posible visualizarlas en la ventana de ejecucion de Pycharm. Ademas los valores de las variables para realizar los calculos se obtienen de las matrices de confusion al final del codigo (ver link de Dri o ejecutar el codigo en Pycharmve)
### 5) Analisis de los Resultados (VER LINK DE DRIVE O EJECUTAR EL CODIGO EN PYCHARM)
Los resultados obtenidos a partir de la evaluacion de ambos modelos indican que la CNN supero significativamente al modelo Random forest en terminos de precision global y rendimiento por clase. en la matriz de confusion de la CNN se observa una mayor cantidad de predicciones correctas para cada una de las clases, especialmente para la categoria “notumor”, donde casi todas las imagenes fueron correctamente clasificadas. Ademas, el número de errores de clasificacion cruzada fue notablemente menor. En cambio, el random forest mostro dificultades mas marcadas, especialmente confundiendo imagenes entre las clases glioma y meningioma, y tambien presento mas errores con las imagenes de la clase pituitary. Este comportamiento se justifica tecnicamente por la naturaleza del modelo: la CNN es capaz de extraer caracteristicas jerarquicas y espaciales de las imagenes, mientras que el Random forest, al trabajar con vectores planos, pierde dicha estructura visual. Por tanto, se concluye que el mejor resultado fue proporcionado por la CNN, ya que logra capturar la complejidad del dominio visual y proporciona una clasificacion mas precisa y confiable, algo esencial en el contexto medico de diagnostico por imagen. La verdad era algo de esperarse ya que las CNN estan especialmente enfocadas al analisis de las imagenes.
### 6) interpretabilidad de los Resultados
En los videos y diferentes paginas de internet se suele mencionar que las CNN son una especie de “cajas negras” debido a su complejidad interna, los resultados obtenidos son interpretables desde un puntos de vista clinicos tanto tecnicos. La visualizacion mediante matrices de confusion facilita la interpretacion de los errores y aciertos por clase, permitiendo entender en que areas el modelo es fuerte y en cuales necesita mejora (como en la vida siempre se puede mejorar :D). Ademas, la distribucion de clases y los resultados metricos ofrecen una vision clara del rendimiento global del modelo. Si quisieramos aumentar la interpretabilidad, podrian aplicarse tecnicas adicionales como mapas de activacion (Grad-CaM, que se menciono en uno de los videos pero no me queda del todo claro como funcionan) para visualizar que regiones de la imagen influyen en la decision del modelo. Sin embargo, incluso sin estas tecnicas avanzadas, la clasificacion por CNN se muestra como una herramienta efectiva y relativamente comprensible cuando se acompaña de buenas visualizaciones y metricas. En este proyecto, la interpretacion de los resultados se logro mediante una adecuada presentacion grafica y el analisis de metricas clave, haciendo que los hallazgos puedan ser comprendidos incluso por profesionales no expertos en ciencia de datos, como medicos o investigadores del area de salud


# CONCLUSION: Las computadoras me dan miedo ya que pueden saber mis gustos por lo que subo o veo en redes :(
### .

.
.
